In [127]:
import pandas as pd
import numpy as np

import gensim
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel, LdaMulticore
from gensim import corpora

import spacy
spacy.load('en', disable=['parser', 'ner'])

!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim

from google.colab import drive

In [128]:
drive.mount("/content/gdrive/")

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [129]:
# Load data
data = pd.read_csv(r'/content/gdrive/My Drive/Datasets/Tweets.csv')

In [124]:
def preprocess_text(tweets, allowed_pos_tags=['NOUN', 'VERB', 'ADV', 'ADJ']):
  cleaned_tweets = list()
  nlp = spacy.load('en_core_web_sm')
  for tweet in tweets:
    # Use simple preprocess to convert to lower case and remove punctuation
    tweet = nlp(' '.join(simple_preprocess(tweet, deacc=True,)))
    # Use spacy to lemmatize and conform token's pos tag is in the allowed list
    cleaned_tweets.append([token.lemma_ for token in tweet if token.pos_ in allowed_pos_tags])
  
  return cleaned_tweets

In [106]:
# Cleaned up text
texts = preprocess_text(data.text)

# Create dictionary
id2word = corpora.Dictionary(texts)

# Create corpus
corpus = [id2word.doc2bow(text) for text in texts]

In [126]:
# Check to confirm the corpus is in bow format
corpus[:3]

[[(0, 1)],
 [(1, 1), (2, 1), (3, 1), (4, 1)],
 [(5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1)]]

**Train LDA Model**

In [144]:
# Build LDA model to create topics
lda_model = LdaMulticore(corpus=corpus, num_topics=8, id2word=id2word, alpha=.4, random_state=1, passes=10, workers=4)

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt

In [145]:
# Get topic coherence
coherence_model = gensim.models.CoherenceModel(model=lda_model, texts=texts, corpus=corpus, dictionary=id2word)
coherence_model.get_coherence()

0.3123169394386769

In [146]:
# Visualize topics
pyLDAvis.enable_notebook()

vis = pyLDAvis.gensim.prepare(topic_model=lda_model, corpus=corpus, dictionary=id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.055782  0.038817       1        1  13.569849
3     -0.060980  0.053358       2        1  13.276798
4      0.212049  0.135161       3        1  12.635748
0     -0.057846 -0.051195       4        1  12.633641
6     -0.108477 -0.067148       5        1  12.289526
7      0.053067 -0.034131       6        1  12.239484
1     -0.094655  0.112140       7        1  12.014821
2      0.112625 -0.187003       8        1  11.340133, topic_info=         Term         Freq        Total Category  logprob  loglift
80      thank  1519.000000  1519.000000  Default  30.0000  30.0000
24     flight  4168.000000  4168.000000  Default  29.0000  29.0000
291  customer   879.000000   879.000000  Default  28.0000  28.0000
335    cancel  1010.000000  1010.000000  Default  27.0000  27.0000
187   service   904.000000   904.000000  Default  26.0000  26.0000
..        ...          ...          ...      ...      ...      ...
29       seat    90.143197   468.078227   Topic8  -5.0563   0.5296
586       see    78.875033   317.747582   Topic8  -5.1898   0.7834
180     leave    79.538845   341.518528   Topic8  -5.1814   0.7197
31      would    68.577523   555.924703   Topic8  -5.3297   0.0842
39         co    66.542012   686.294388   Topic8  -5.3599  -0.1567

[613 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
449       2  0.046122  absolute
449       3  0.876316  absolute
449       7  0.046122  absolute
2653      1  0.789419   abysmal
217       3  0.025215   account
...     ...       ...       ...
73        4  0.074799       yet
73        5  0.598396       yet
73        6  0.059840       yet
73        7  0.014960       yet
73        8  0.082279       yet

[1987 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 4, 5, 1, 7, 8, 2, 3])